In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [2]:
page = requests.get('https://talksport.com/football/557770/most-valuable-players-football-neymar-cristiano-ronaldo-mohamed-salah/')
soup = BeautifulSoup(page.content,'html.parser')

In [3]:
main_content = soup.find(id='main-content')
players = main_content.find_all('h3',attrs={"class":""})

In [4]:
player_list = []
players_list = []
for i in range(0,len(players)):
    if (str(players[i]).find('talksport') != -1):
#         print(players[i])
        rank = (players[i].contents[0][0:players[i].contents[0].find('.')])
        name = (players[i].contents[0][players[i].contents[0].find('.')+2:players[i].contents[0].find('(')-1])
        team = (players[i].contents[1].contents[0])
        value = (players[i].contents[2][players[i].contents[2].find('£'):])
        player_list = [rank,name,team,value]
        players_list.append(player_list)
    else:
        player_dat = players[i].contents[0]
#         print(player_dat)
        rank = (player_dat[0:player_dat.find('.')])
        name = (player_dat[player_dat.find('.')+2:player_dat.find('(')-1])
        team = (player_dat[player_dat.find('(')+1:player_dat.find(')')])
        value = (player_dat[player_dat.find('£'):])
        player_list = [rank,name,team,value]
        players_list.append(player_list)

In [5]:
top_50_players_df = pd.DataFrame.from_records(players_list,columns=['Rank','Player','Team','Value in GBP'])

In [22]:
unique_teams = top_50_players_df['Team'].unique()

In [23]:
import os

In [24]:
path = '..\\Consolidated Data'

files = []
for r, d, f in os.walk(path):
    for file in f:
        if '2019.csv' in file:
            files.append(file)
full_df = pd.DataFrame(columns=('Team','League'))
for f in files:
    file_name = '..\\Consolidated Data\\'+f
    temp_df = pd.read_csv(file_name,usecols = ['Team'])
    temp_df['League'] = f[0:f.find('_2019')]
    full_df = full_df.append(temp_df)    

In [25]:
unique_df = pd.DataFrame(columns=['Team','League'])
unique_teams = full_df['Team'].unique()

In [26]:
x=0
for i in unique_teams:
    try:
        unique_df.loc[x]= [i,full_df[full_df['Team']==i]['League'].values[0]]
        x=x+1
    except:
        unique_df.loc[x]= [i,'League not found']
        x=x+1

In [27]:
unique_df = unique_df.set_index('Team')

In [12]:
# unique_df.loc['Bayer Leverkusen']['League'] ='Bundesliga'
# unique_df.loc['Manchester City']['League'] ='English_Premiership'
# unique_df.loc['Manchester United']['League'] ='English_Premiership'
# unique_df.loc['Tottenham Hotspur']['League'] ='English_Premiership'
# unique_df.loc['Paris Saint-Germain']['League'] ='Ligue 1'

In [28]:
unique_df=unique_df.reset_index()

In [14]:
final_df = pd.merge(top_50_players_df,unique_df,on='Team')

In [15]:
final_series = final_df.groupby('League').count().Player

In [16]:
final_df = final_series.to_frame()

In [17]:
final_df.to_csv('..\\Consolidated Data\\Top 50 Players Consildation.csv')

In [9]:
top_50_players_df.to_csv('..\\Consolidated Data\\Top 50 Players Club Level.csv')